# Grocery Store Suggestions System

---




In the daily life, We use to forget something and realize after arriving at home. To solve this problem and increase the client satisfaction, Our team suggest a recommendation system for online and off-line store in this project. We aim to help our customer shop in a convenient environment by suggesting the related producted based on the actual data presendted by bonÀrea. 

This project has just started in *Google Colab®* and it has started with open and explore data from csv files.

As *Google Collab®* doen't has Spark installed by default, next block prepare the environment to can work with Spark. Also as this platform is efimer the data is managed with a mounted volume to *Google Drive®*. 

In [ ]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa

In [ ]:
try:
  # Only it runs in google collab
  from google.colab import drive # if this package don't exist we aren't con google colab
  import os
  # Install spark and its requirements
  !apt-get install openjdk-8-jdk-headless -qq > /dev/null
  !wget -nc -q https://dlcdn.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
  !tar xf spark-3.2.0-bin-hadoop3.2.tgz
  !pip install -q pyspark
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
  os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"
  drive.mount('/content/drive')
  base_path = '/content/drive/MyDrive/data/' # Google Drive
except:
  # It runs in local 
  base_path = ''


     |████████████████████████████████| 281.3 MB 43 kB/s 
     |████████████████████████████████| 198 kB 59.8 MB/s 
Mounted at /content/drive


Importing libraries to work with Spark and initializate a SparkSession.

In [ ]:
# Import Spark libs
from pyspark.sql import SparkSession
from IPython.display import display, HTML
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col


# Initiate SparkSession
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.sql.execution.arrow.enabled", "true") \
    .getOrCreate()

The path of files. This uses a base path in order to be able to work this Jupyter notebook wherever (at moment in colab and local).

In [ ]:
# File Path
file_products = base_path + 'ARTICLES.csv'
file_families = base_path + 'AECOC.csv'
file_sells = base_path + 's20.csv'

#### Products file

In [ ]:
# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ";"

df_product = spark.read.format('csv') \
  .option("charset", "ISO-8859-1") \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_products)
df_product.limit(5).toPandas()

,ARTICLE,DESCRIPCIO,SECTOR,SECCIO,FAMILIA,DESC
0,10000,LIMPIACRISTALES CON,3,1,6,03*01*06
1,10001,LIMPIACRISTALES RECA,3,1,6,03*01*06
2,10002,LIMPIACRISTALES BONA,3,1,6,03*01*06
3,10003,LIMPIACRISTALES BONA,3,1,6,03*01*06
4,10004,FREGASUELOS CAG 1 L,3,1,6,03*01*06


It make a projection of this file with selected columns and normalized its name.

In [ ]:
print ("total products: ", df_product.count())
df_product = df_product.selectExpr('ARTICLE as product_id','DESC as family_id', 'DESCRIPCIO as product_desc' )
df_product.limit(5).toPandas()


total products:  72200


,product_id,family_id,product_desc
0,10000,03*01*06,LIMPIACRISTALES CON
1,10001,03*01*06,LIMPIACRISTALES RECA
2,10002,03*01*06,LIMPIACRISTALES BONA
3,10003,03*01*06,LIMPIACRISTALES BONA
4,10004,03*01*06,FREGASUELOS CAG 1 L


#### Families file

In [ ]:
# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ";"

df_families = spark.read.format('csv') \
  .option("charset", "ISO-8859-1") \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_families)

df_families.limit(5).toPandas()

,FAMILIA,DESCRIPCIO.SECTOR,DESCRIPCIO.SECCIO,DESC.FAMILIA,_c4,_c5,_c6,_c7
0,00*00*00,Desconocido,,Desconeguda,None,None,None,None
1,01*01*01,Alimentacion y Bebidas,ALIMENTACIÓN SECA,Aceites,None,None,None,None
2,01*01*02,Alimentacion y Bebidas,ALIMENTACIÓN SECA,Cafés y sucedáneos,None,None,None,None
3,01*01*03,Alimentacion y Bebidas,ALIMENTACIÓN SECA,Infusiones,None,None,None,None
4,01*01*04,Alimentacion y Bebidas,ALIMENTACIÓN SECA,Chocolates,None,None,None,None


In [ ]:
schema = StructType([
    StructField("family_id", StringType(), True),
    StructField("sector_desc", StringType(), True),
    StructField("section_desc", StringType(), True),
    StructField("family_desc", StringType(), True),
    StructField("not_use1", StringType(), True),
    StructField("not_use2", StringType(), True),
    StructField("not_use3", StringType(), True),
    StructField("not_use4", StringType(), True)
    ])

df_families = spark.read.format('csv') \
  .schema(schema) \
  .option("charset", "ISO-8859-1") \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_families)


print ("total families: ", df_families.count())

df_families.select(col('family_id'), col('family_desc') ).limit(5).toPandas()


total families:  816


,family_id,family_desc
0,00*00*00,Desconeguda
1,01*01*01,Aceites
2,01*01*02,Cafés y sucedáneos
3,01*01*03,Infusiones
4,01*01*04,Chocolates


In [ ]:
# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = ";"

schema = StructType([
    StructField("invoice_id", StringType(), True),
    StructField("product_id", IntegerType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("amount", IntegerType(), True),
    StructField("checkout", IntegerType(), True),
    StructField("date", IntegerType(), True),
    StructField("hour", IntegerType(), True),])

df_sells = spark.read.format('csv') \
  .schema(schema) \
  .option("charset", "ISO-8859-1") \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_sells)

df_sells.limit(5).toPandas()


,invoice_id,product_id,quantity,amount,checkout,date,hour
0,2027-T0132C01-000001,6252,4000000,419,1,18995,31737
1,2027-T0132C01-000001,4465,1000000,148,1,18995,31737
2,2027-T0132C01-000001,4465,1000000,132,1,18995,31737
3,2027-T0132C01-000001,4465,1000000,131,1,18995,31737
4,2027-T0132C01-000001,4465,1000000,130,1,18995,31737


In [ ]:
from datetime import datetime,timedelta

print("total sells: ", df_sells.count())

StartDate = "12/31/1967"
Date = datetime.strptime(StartDate, "%m/%d/%Y")

df_sells = df_sells.rdd \
  .map(lambda x: (x[0],x[1],x[2]/1000000,x[3]/100,x[4],Date.replace(hour=0, minute=0, second=0, microsecond =0)+timedelta(days=x[5], seconds=x[6]))) \
  .toDF(['invoice_id','product_id', 'quantity','amount','checkout','datetime'])

df_sells.limit(5).toPandas()

total sells:  1156642


,invoice_id,product_id,quantity,amount,checkout,datetime
0,2027-T0132C01-000001,6252,4.0,4.19,1,2020-01-02 08:48:57
1,2027-T0132C01-000001,4465,1.0,1.48,1,2020-01-02 08:48:57
2,2027-T0132C01-000001,4465,1.0,1.32,1,2020-01-02 08:48:57
3,2027-T0132C01-000001,4465,1.0,1.31,1,2020-01-02 08:48:57
4,2027-T0132C01-000001,4465,1.0,1.30,1,2020-01-02 08:48:57


In [ ]:
products_to_avoid = [
                     8055, # plastic bag
                     9117, # plastic bag
                     8419, # raffia bag
                     ]
df_sells = df_sells[~df_sells['product_id'].isin(products_to_avoid)]

#Training

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql.functions import monotonically_increasing_id, regexp_replace
from pyspark.sql import functions as F

In [ ]:
df_sells_w_product_desc = df_sells.join(df_product,on="product_id")
#df_sells_w_product_desc.limit(5).toPandas()

In [ ]:
df_sells_by_invoice_id = df_sells_w_product_desc.groupBy("invoice_id", "datetime").agg(F.collect_list("product_id").alias("product_id"), F.collect_list("quantity").alias("quantities"), F.collect_list("amount").alias("amounts"))
df_sells_by_invoice_id = df_sells_by_invoice_id.filter(F.size(F.col("product_id")) > 2)
#df_sells_by_invoice_id.limit(5).toPandas()

In [ ]:
df_sells_before_split = df_sells_by_invoice_id.withColumn("new", F.arrays_zip("product_id", "quantities", "amounts"))\
       .withColumn("new", F.explode("new"))\
       .select("invoice_id", "datetime", F.col("new.product_id").alias("product_id"), F.col("new.quantities").alias("quantity"), F.col("new.amounts").alias("amount"))
df_sells_before_split = df_sells_before_split.na.drop()
df_sells_before_split.limit(10).toPandas()

,invoice_id,datetime,product_id,quantity,amount
0,2027-T0132C01-000003,2020-01-02 09:10:44,2412,1.0,1.27
1,2027-T0132C01-000003,2020-01-02 09:10:44,2402,1.0,1.20
2,2027-T0132C01-000003,2020-01-02 09:10:44,6183,1.0,2.48
3,2027-T0132C01-000003,2020-01-02 09:10:44,2408,1.0,0.87
4,2027-T0132C01-000003,2020-01-02 09:10:44,3227,1.0,3.73
5,2027-T0132C01-000003,2020-01-02 09:10:44,7587,1.0,1.99
6,2027-T0132C01-000004,2020-01-02 09:13:08,8286,1.0,0.67
7,2027-T0132C01-000004,2020-01-02 09:13:08,6170,1.0,2.23
8,2027-T0132C01-000004,2020-01-02 09:13:08,5880,1.0,0.90
9,2027-T0132C01-000004,2020-01-02 09:13:08,6631,1.0,2.20


In [ ]:
(training, test) = df_sells_before_split.randomSplit([0.8, 0.2])
#training.limit(5).toPandas()

In [ ]:
training_by_product3 = training.groupBy("invoice_id","product_id").sum("quantity")
training_by_product3 = training_by_product3.withColumnRenamed("sum(quantity)", "sum_quantity")
training_by_product3.limit(10).toPandas()

,invoice_id,product_id,sum_quantity
0,2027-T0132C01-000125,2111,1.0
1,2027-T0132C01-000226,7302,1.0
2,2027-T0132C01-000251,7056,2.0
3,2027-T0132C01-000312,7551,6.0
4,2027-T0132C01-000355,5916,2.0
5,2027-T0132C01-000416,6166,1.0
6,2027-T0132C01-000417,5845,1.0
7,2027-T0132C01-000423,85488,1.0
8,2027-T0132C01-000568,6215,1.0
9,2027-T0132C01-000692,1608,1.0


In [ ]:
training_by_product4 = training_by_product3.withColumn('product_id', training_by_product3["product_id"].cast(IntegerType()))
training_by_product5 = training_by_product4.withColumn('invoice_id', regexp_replace('invoice_id', '2027-T0132C01-', ''))
training_by_product_final = training_by_product5.withColumn('invoice_id', training_by_product5["invoice_id"].cast(IntegerType()))
training_by_product_final = training_by_product_final.na.drop()
training_by_product_final.show()

+----------+----------+------------+
|invoice_id|product_id|sum_quantity|
+----------+----------+------------+
|       125|      2111|         1.0|
|       226|      7302|         1.0|
|       251|      7056|         2.0|
|       312|      7551|         6.0|
|       355|      5916|         2.0|
|       416|      6166|         1.0|
|       417|      5845|         1.0|
|       423|     85488|         1.0|
|       568|      6215|         1.0|
|       692|      1608|         1.0|
|       795|      8373|         2.0|
|       927|      7659|         1.0|
|      1080|      6497|         1.0|
|      1233|      4487|         1.0|
|      1397|      5697|         1.0|
|      1501|      5921|         2.0|
|      1509|      7550|         1.0|
|      1627|      1648|         1.0|
|      1797|      8232|         1.0|
|      1830|      5214|         1.0|
+----------+----------+------------+
only showing top 20 rows



Because the rating matrix is derived from another source of information (i.e. it is inferred from other signals), we set implicitPrefs to True to get better results

In [ ]:
# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, implicitPrefs=True, userCol="invoice_id", itemCol="product_id", ratingCol="sum_quantity",
          coldStartStrategy="drop")
model = als.fit(training_by_product_final)

#Arrange test dataset

In [ ]:
test_by_product_complete = test.groupBy("invoice_id", "datetime").agg(F.collect_list("product_id").alias("product_id"), F.collect_list("quantity").alias("quantities"), F.collect_list("amount").alias("amounts"))
#test_by_product_complete.show()

In [ ]:
sells_test_results = test_by_product_complete.withColumn('product_expected', F.col('product_id')[0])
sells_test_results = sells_test_results.withColumn('product_without_expected', F.expr("slice(product_id, 2, SIZE(product_id))"))
sells_test_results = sells_test_results.withColumn('quantities_without_expected', F.expr("slice(quantities, 2, SIZE(quantities))"))
sells_test_results = sells_test_results.withColumn('amounts_without_expected', F.expr("slice(amounts, 2, SIZE(amounts))"))
sells_test_results = sells_test_results.withColumn('invoice_id', regexp_replace('invoice_id', '2027-T0132C01-', ''))
sells_test_results = sells_test_results.withColumn('invoice_id', sells_test_results["invoice_id"].cast(IntegerType()))
#sells_test_results.show()

In [ ]:
test_by_product_incomplete = sells_test_results[['invoice_id', 'datetime', 'product_without_expected', 'quantities_without_expected', 'amounts_without_expected']]
#test_by_product_incomplete.show()

In [ ]:
test_by_product_incomplete_explode = test_by_product_incomplete.withColumn("new", F.arrays_zip("product_without_expected", "quantities_without_expected", "amounts_without_expected"))\
       .withColumn("new", F.explode("new"))\
       .select("invoice_id", "datetime", F.col("new.product_without_expected").alias("product_id"), F.col("new.quantities_without_expected").alias("quantity"), F.col("new.amounts_without_expected").alias("amount"))
test_by_product_incomplete_explode = test_by_product_incomplete_explode.na.drop()
#test_by_product_incomplete_explode.show()

In [ ]:
test_by_product_incomplete_groupby = test_by_product_incomplete_explode.groupBy("invoice_id","product_id").sum("quantity")
test_by_product_incomplete_groupby = test_by_product_incomplete_groupby.withColumnRenamed("sum(quantity)", "sum_quantity")
#test_by_product_incomplete_groupby.show()

In [ ]:
test_by_product_final = test_by_product_incomplete_groupby.withColumn('product_id', test_by_product_incomplete_groupby["product_id"].cast(IntegerType()))
test_by_product_final = test_by_product_final.filter(test_by_product_final['sum_quantity']>0)
test_by_product_final.limit(10).toPandas()

,invoice_id,product_id,sum_quantity
0,1712,8921,1.0
1,1966,8198,1.0
2,2512,8368,1.0
3,3072,2099,1.0
4,3339,7062,1.0
5,3533,7056,1.0
6,5256,5974,1.0
7,5994,7755,1.0
8,6181,4483,1.0
9,7043,5214,1.0


#Predictions for test dataset

In [ ]:
pred = model.recommendForUserSubset(test_by_product_final, 1)
pred.show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+----------+--------------------+
|invoice_id|     recommendations|
+----------+--------------------+
|         1| [{6252, 0.9225558}]|
|        12|[{6252, 0.68864095}]|
|        13|[{4493, 0.34657037}]|
|        19|[{6253, 0.81527174}]|
|        31|[{2111, 0.12528391}]|
|        47|[{7617, 0.19406353}]|
|        48|[{5856, 0.19915876}]|
|        88|[{8127, 0.06910945}]|
|       112| [{7550, 0.6651385}]|
|       127|[{5954, 0.0467335...|
|       142|[{6252, 0.0363418...|
|       146| [{2111, 0.6564643}]|
|       148|[{8202, 0.0618579...|
|       155|[{3060, 0.07463005}]|
|       168|[{2111, 0.0167379...|
|       175|[{5856, 0.26843783}]|
|       178| [{2111, 0.6873528}]|
|       185|[{6899, 0.0314922...|
|       189|[{5954, 0.13910255}]|
|       217| [{8202, 0.1916081}]|
+----------+--------------------+
only showing top 20 rows



#Compare predictions

In [ ]:
sells_test_results_w_recommendation = sells_test_results.join(pred,on="invoice_id")
sells_test_results_w_recommendation = sells_test_results_w_recommendation.withColumn('recommendations', F.col('recommendations').cast(StringType()))
sells_test_results_w_recommendation = sells_test_results_w_recommendation.withColumn('recommendations', F.regexp_extract('recommendations', r'\{([0-9]+).*',1))
#sells_test_results_w_recommendation.show()

In [ ]:
sells_test_results_w_recommendation = sells_test_results_w_recommendation.withColumn('success_recomended', F.col('recommendations') == F.col('product_expected'))
#sells_test_results_w_recommendation.show()

In [ ]:
ok = sells_test_results_w_recommendation.filter(F.col('success_recomended') == 'true')
ok.show()

+----------+-------------------+--------------------+--------------------+--------------------+----------------+------------------------+---------------------------+------------------------+---------------+------------------+
|invoice_id|           datetime|          product_id|          quantities|             amounts|product_expected|product_without_expected|quantities_without_expected|amounts_without_expected|recommendations|success_recomended|
+----------+-------------------+--------------------+--------------------+--------------------+----------------+------------------------+---------------------------+------------------------+---------------+------------------+
|       223|2020-01-02 13:28:13|  [5856, 6816, 6816]|     [1.0, 1.0, 1.0]|  [1.05, 1.85, 1.89]|            5856|            [6816, 6816]|                 [1.0, 1.0]|            [1.85, 1.89]|           5856|              true|
|       599|2020-01-03 12:10:08|  [5954, 6461, 8167]|     [1.0, 1.0, 1.0]|  [4.35, 0.54, 0.21]| 

In [ ]:
nb_ok = ok.count()
nb_ok

1346

In [ ]:
nb_ok = ok.count()
total = sells_test_results_w_recommendation.count()
percent = 100 * nb_ok / total
percent

3.2057541620025245

# Conclusion

The ALS algorithm works with a rating column, but in our case, we dont have one. So we use an artificial one, the quantity of each product for each purchase and we set *implicitPrefs* to true. Change this parameter gives us better results.

Work with the price payed for each product as the rating colum is also a option that has to be seriously considered.